# VacationPy
----

### Note


In [1]:
%matplotlib inline

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
weather_data = "../output_data/weather_data.csv"

In [3]:
# create dataframe and drop any empty cells
output_df = pd.read_csv(weather_data)
output_df.dropna(inplace = True)
output_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Feels Like,Wind Speed
0,Upernavik,0,GL,1598655460,71,72.79,-56.15,4.64,4.64,-2.30,7.04
1,Norman Wells,90,CA,1598655397,66,65.28,-126.83,11.00,11.00,6.98,4.10
2,Rāmgarh,7,IN,1598655037,87,23.63,85.52,22.88,22.88,25.20,2.38
3,Camocim,14,BR,1598655037,72,-2.90,-40.84,26.10,26.10,24.28,8.33
4,Umm Kaddadah,71,SD,1598655037,55,13.60,26.69,28.30,28.30,30.31,1.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# access maps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Lat & lng values as locations variable and humidity as weight
locations = output_df[["Lat", "Lng"]]
humidity = output_df["Humidity"].astype(int)

In [6]:
# plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                point_radius=4)

#add layer
fig.add_layer(heat_layer)

#show figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * A max temperature lower than 27C(80F) degrees but higher than 21C(70F).
    * Wind speed less than 10 mph.
    * Zero cloudiness.
* Drop any rows will null values. The weather should be ideal!

In [7]:
# setting weather conditions in new df
bestweather_df = output_df.loc[(output_df['Max Temp']>21) & (output_df['Max Temp']<27)
                      & (output_df["Wind Speed"]<10) & (output_df["Cloudiness"]==0)]
bestweather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Feels Like,Wind Speed
29,Ardakān,0,IR,1598655464,15,32.31,54.02,25.00,25.00,20.39,3.10
37,Arraial do Cabo,0,BR,1598655466,78,-22.97,-42.03,22.00,22.00,20.10,6.70
72,Jalu,0,LY,1598655471,60,29.03,21.55,26.03,26.03,25.25,4.89
85,Tiznit Province,0,MA,1598655473,49,29.58,-9.50,24.17,24.17,23.35,2.40
87,Tabriz,0,IR,1598655473,38,38.08,46.29,23.00,23.00,21.04,2.10
101,Marsá Maţrūḩ,0,EG,1598655475,65,31.35,27.25,26.00,26.00,27.37,2.60
107,Lagoa,0,PT,1598655476,94,39.05,-27.98,23.42,23.42,23.58,6.79
155,Podgorica,0,ME,1598655483,60,42.44,19.26,24.00,24.00,23.72,3.10
171,Safidon,0,IN,1598655486,77,29.42,76.67,26.66,26.66,29.55,2.81
231,Caravelas,0,BR,1598655497,83,-17.71,-39.25,21.48,21.48,23.17,1.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

### START OF API CALL.  DON'T RUN AGAIN.

In [9]:
# add hotel name to df
hotel_df = bestweather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
    
# get parameters ready
target_type = "hotel"
target_search = "hotels"
target_radius = 5000

# Make parameters dictionary
params = {
    "radius": target_radius,
    "keyword": target_search,
    "type": target_type,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat & lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # add location to params dict
    params['location'] = f"{lat},{lng}"
    
    # get base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # save results in a variable results
    results = response['results']
    
    # collect avaialbe hotel names and place it in Hotel Name column in hotel_df
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


### END OF API CALL.  DON'T RUN AGAIN.

In [10]:
# verify that hotels are in Hotel Name column
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Feels Like,Wind Speed,Hotel Name
0,Ardakān,0,IR,1598655464,15,32.31,54.02,25.00,25.00,20.39,3.10,Kheshtomah Ecolodge
1,Arraial do Cabo,0,BR,1598655466,78,-22.97,-42.03,22.00,22.00,20.10,6.70,Orlanova Hotel
2,Jalu,0,LY,1598655471,60,29.03,21.55,26.03,26.03,25.25,4.89,Jalu Hotel
3,Tiznit Province,0,MA,1598655473,49,29.58,-9.50,24.17,24.17,23.35,2.40,
4,Tabriz,0,IR,1598655473,38,38.08,46.29,23.00,23.00,21.04,2.10,Sina Hotel
5,Marsá Maţrūḩ,0,EG,1598655475,65,31.35,27.25,26.00,26.00,27.37,2.60,Porto Matrouh Hotel
6,Lagoa,0,PT,1598655476,94,39.05,-27.98,23.42,23.42,23.58,6.79,Graciosa Resort - Biosphere Island Hotel
7,Podgorica,0,ME,1598655483,60,42.44,19.26,24.00,24.00,23.72,3.10,CentreVille Hotel & Experiences
8,Safidon,0,IN,1598655486,77,29.42,76.67,26.66,26.66,29.55,2.81,OYO 69884 Shri Balaji Guest House
9,Caravelas,0,BR,1598655497,83,-17.71,-39.25,21.48,21.48,23.17,1.87,Pousada Encanto Abrolhos


### Saved to csv
hotels.to_csv("../output_data/hotels1.csv", index=False, header=True)

hotels.to_csv("../output_data/hotels.csv", index=False, header=True)

In [18]:
# replace blank hotel names with NaN
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [19]:
# drop NaNs and verify change
hotel_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Feels Like,Wind Speed,Hotel Name
0,Ardakān,0,IR,1598655464,15,32.31,54.02,25.00,25.00,20.39,3.10,Kheshtomah Ecolodge
1,Arraial do Cabo,0,BR,1598655466,78,-22.97,-42.03,22.00,22.00,20.10,6.70,Orlanova Hotel
2,Jalu,0,LY,1598655471,60,29.03,21.55,26.03,26.03,25.25,4.89,Jalu Hotel
4,Tabriz,0,IR,1598655473,38,38.08,46.29,23.00,23.00,21.04,2.10,Sina Hotel
5,Marsá Maţrūḩ,0,EG,1598655475,65,31.35,27.25,26.00,26.00,27.37,2.60,Porto Matrouh Hotel
6,Lagoa,0,PT,1598655476,94,39.05,-27.98,23.42,23.42,23.58,6.79,Graciosa Resort - Biosphere Island Hotel
7,Podgorica,0,ME,1598655483,60,42.44,19.26,24.00,24.00,23.72,3.10,CentreVille Hotel & Experiences
8,Safidon,0,IN,1598655486,77,29.42,76.67,26.66,26.66,29.55,2.81,OYO 69884 Shri Balaji Guest House
9,Caravelas,0,BR,1598655497,83,-17.71,-39.25,21.48,21.48,23.17,1.87,Pousada Encanto Abrolhos
10,São Mateus,0,BR,1598655519,90,-18.72,-39.86,21.22,21.22,23.41,1.82,ibis Styles Sao Mateus


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))